# Question 1. Running Elastic

In [1]:
# Elastic Search Index

from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
es_client.info()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Q2. Indexing the data

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-course-questions"

es_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-course-questions'})

In [10]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:26<00:00, 35.51it/s]


# Q3. Searching

In [27]:
def elastic_search_to_score(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])

    return result_docs


In [43]:
query = "How do I execute a command in a running docker container?"

elastic_search_to_score(query)

[84.050095, 51.04628, 49.938507, 45.275463, 45.255775]

# Q4. Filtering

In [37]:
def elastic_search(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [38]:
questions = ["How do I debug a docker container?",
"How do I copy files from a different folder into docker container’s working directory?",
"How do Lambda container images work?",
"How can I annotate a graph?"]

In [39]:
answers = []

for q in questions:
    answers.append(elastic_search(q))

In [57]:
print(answers[2])

[{'text': 'I wanted to understand how lambda container images work in depth and how lambda functions are initialized, for this reason, I found the following documentation\nhttps://docs.aws.amazon.com/lambda/latest/dg/images-create.html\nhttps://docs.aws.amazon.com/lambda/latest/dg/runtimes-api.html\nAdded by Alejandro aponte', 'section': '9. Serverless Deep Learning', 'question': 'How do Lambda container images work?', 'course': 'machine-learning-zoomcamp'}, {'text': 'Tim from BentoML has prepared a dedicated video tutorial wrt this use case here:\nhttps://www.youtube.com/watch?v=7gI1UH31xb4&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR&index=97\nKonrad Muehlberg', 'section': 'Miscellaneous', 'question': 'How to pass BentoML content / docker container to Amazon Lambda', 'course': 'machine-learning-zoomcamp'}, {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is 

# Q5. Building a prompt

In [46]:
def build_prompt(question,search_results):
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=question, context=context).strip()
    print("Length of the Prompt : ",len(prompt))
    
    return prompt

In [47]:
query = "How do I execute a command in a running docker container?"

search_results = elastic_search(query)
prompt = build_prompt(query,search_results)

Length of the Prompt :  1660


# Q6. Tokens

In [51]:
# pip install tiktoken

In [52]:
import tiktoken

In [55]:
tokenizer = tiktoken.encoding_for_model("gpt-4o")

In [56]:
tokens = tokenizer.encode(prompt)
num_tokens = len(tokens)

# Output the result
print(f"Number of tokens: {num_tokens}")

Number of tokens: 361


# Bonus: generating the answer (ungraded)

In [62]:
import os
from groq import Groq
from dotenv import load_dotenv

def llm(prompt):
    # load environment variables
    load_dotenv()
    
    # create client calling Groq class
    client = Groq(api_key=os.getenv('GROQ_API_KEY'))
    
    # create a query
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )
    
    # print the response
    return response.choices[0].message.content

In [63]:
answer = llm(prompt)
print(answer)

According to the context, to execute a command in a running Docker container, you need to:

1. Find the container ID by running `docker ps`.
2. Execute the command using `docker exec -it <container-id> bash`.

This will open a new interactive bash session within the running Docker container, allowing you to execute commands.


# Bonus: calculating the costs (ungraded)

In [64]:
# Define the given data
tokens_per_request_input = 150
tokens_per_request_output = 250
number_of_requests = 1000
cost_per_1k_tokens_input = 0.005
cost_per_1k_tokens_output = 0.015

# Calculate the total tokens for 1000 requests
total_tokens_input = tokens_per_request_input * number_of_requests
total_tokens_output = tokens_per_request_output * number_of_requests

# Calculate the cost of the input tokens
cost_input_tokens = (total_tokens_input / 1000) * cost_per_1k_tokens_input

# Calculate the cost of the output tokens
cost_output_tokens = (total_tokens_output / 1000) * cost_per_1k_tokens_output

# Calculate the total cost
total_cost = cost_input_tokens + cost_output_tokens

# Output the result
print(f"Total cost to run {number_of_requests} requests: ${total_cost:.2f}")


Total cost to run 1000 requests: $4.50
